In [1]:
!pip install nltk scikit-learn pandas flask pyngrok joblib wordcloud

In [2]:
# import os
# import re
# import string
# import numpy as np
# import pandas as pd
# import nltk
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer, PorterStemmer
# import requests
# import tarfile
# from io import BytesIO
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_selection import SelectKBest, chi2
# from sklearn.svm import LinearSVC
# from sklearn.metrics import accuracy_score
# import joblib
# from flask import Flask, request, render_template_string, jsonify
# from pyngrok import ngrok
# import threading
# import time
# import psutil
# import shutil
# import logging

# # Set up logging
# logging.basicConfig(level=logging.INFO)

# # Download NLTK data
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('punkt_tab')

# # Download and extract dataset
# url = "http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz"
# output_csv = "data_set.csv"
# print("Downloading dataset...")
# response = requests.get(url)
# if response.status_code != 200:
#     raise Exception("Failed to download the dataset")
# print("Extracting dataset...")
# tar = tarfile.open(fileobj=BytesIO(response.content), mode="r:gz")
# tar.extractall(path="movie_reviews")
# tar.close()

# # Load reviews and labels
# reviews = []
# labels = []
# base_dir = "movie_reviews/txt_sentoken"
# pos_dir = os.path.join(base_dir, "pos")
# for filename in os.listdir(pos_dir):
#     if filename.endswith(".txt"):
#         with open(os.path.join(pos_dir, filename), "r", encoding="utf-8") as file:
#             review = file.read().strip()
#             reviews.append(review)
#             labels.append(1)
# neg_dir = os.path.join(base_dir, "neg")
# for filename in os.listdir(neg_dir):
#     if filename.endswith(".txt"):
#         with open(os.path.join(neg_dir, filename), "r", encoding="utf-8") as file:
#             review = file.read().strip()
#             reviews.append(review)
#             labels.append(0)

# # Create CSV file
# print("Creating CSV file...")
# data = {"review": reviews, "sentiment": labels}
# df = pd.DataFrame(data)
# df.to_csv(output_csv, index=False, encoding="utf-8")
# print(f"Dataset saved to {output_csv}")
# shutil.rmtree("movie_reviews")
# print("Cleaned up temporary files")

# # Text preprocessing functions
# def remove_punctuation(text):
#     return text.translate(str.maketrans('', '', string.punctuation))

# def normalize_text(text):
#     normalization_map = {
#         'u': 'you',
#         'ur': 'your',
#         'r': 'are',
#         'b4': 'before',
#         'gr8': 'great',
#         'gud': 'good',
#         'pls': 'please',
#         'plz': 'please',
#         'thx': 'thanks',
#         'l8r': 'later',
#         'msg': 'message',
#         'btw': 'by the way',
#         'idk': 'i do not know',
#         'imo': 'in my opinion'
#     }
#     words = text.split()
#     return ' '.join([normalization_map.get(word, word) for word in words])

# def stem_text(text):
#     stemmer = PorterStemmer()
#     stop_words = set(stopwords.words('english')) - {'was'}
#     tokens = text.split()
#     return ' '.join([stemmer.stem(token) for token in tokens if token not in stop_words])

# def remove_word_duplicates(text):
#     words = word_tokenize(text)
#     seen = set()
#     new_words = []
#     for word in words:
#         if word.lower() not in seen:
#             seen.add(word.lower())
#             new_words.append(word)
#     return ' '.join(new_words)

# def preprocess_text(text):
#     text = text.lower()
#     text = remove_punctuation(text)
#     text = normalize_text(text)
#     text = ' '.join([word for word in text.split() if word not in (set(stopwords.words('english')) - {'was'})])
#     text = remove_word_duplicates(text)
#     lemmatizer = WordNetLemmatizer()
#     text = ' '.join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text)])
#     text = stem_text(text)
#     return text

# # Preprocess data
# df['review'] = df['review'].apply(preprocess_text)
# df['sentiment'] = df['sentiment'].astype(str)

# # Split data
# X = df['review'].astype(str).tolist()
# y = df['sentiment']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Vectorize and select features
# vectorizer = TfidfVectorizer(max_features=10000)
# X_train_tfidf = vectorizer.fit_transform(X_train)
# X_test_tfidf = vectorizer.transform(X_test)

# selector = SelectKBest(chi2, k=5000)
# X_train_tfidf = selector.fit_transform(X_train_tfidf, y_train)
# X_test_tfidf = selector.transform(X_test_tfidf)

# # Train model
# model = LinearSVC(C=1, random_state=42, class_weight='balanced', max_iter=10000)
# model.fit(X_train_tfidf, y_train)

# # Evaluate model
# y_pred = model.predict(X_test_tfidf)
# print(f"Test Accuracy: {accuracy_score(y_test, y_pred)}")

# # Debug vocabulary and feature weights
# print("Vocabulary contains 'good':", "good" in vectorizer.vocabulary_)
# print("Vocabulary contains 'movi':", "movi" in vectorizer.vocabulary_)
# selected_features = vectorizer.get_feature_names_out()[selector.get_support()]
# print("Selected features contain 'good':", "good" in selected_features)
# print("Selected features contain 'movi':", "movi" in selected_features)
# feature_weights = model.coef_[0]
# feature_importance = dict(zip(selected_features, feature_weights))
# print("Weight for 'good':", feature_importance.get("good", "Not found"))
# print("Weight for 'movi':", feature_importance.get("movi", "Not found"))

# # Save model, vectorizer, and selector
# joblib.dump(model, 'linear_svc_model.pkl')
# joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
# joblib.dump(selector, 'chi2_selector.pkl')
# print("Model, vectorizer, and selector saved.")

# # HTML template
# html_template = """
# <!DOCTYPE html>
# <html lang="en">
# <head>
#     <meta charset="UTF-8">
#     <meta name="viewport" content="width=device-width, initial-scale=1.0">
#     <title>Sentiment Analysis</title>
#     <link rel="stylesheet" href="/static/style.css">
# </head>
# <body>
#     <div class="header">
#         <h2>Sentiment Analysis of Movie Reviews</h2>
#     </div>
#     <div class="parent">
#         <div class="collection">
#             <p>Please enter your text in English for analysis</p>
#             <form onsubmit="analyzeReview(event)">
#                 <textarea id="reviewInput" placeholder="Type here..." cols="70" rows="10"></textarea>
#                 <button type="submit">Analyze</button>
#             </form>
#             <div id="result" class="result"></div>
#             <div id="processed" class="processed"></div>
#         </div>
#     </div>

#     <script>
#         // Auto-resize textarea
#         const textarea = document.getElementById('reviewInput');
#         textarea.addEventListener('input', function () {
#             this.style.height = 'auto';
#             this.style.height = `${this.scrollHeight}px`;
#         });

#         // Handle form submission and send request to backend
#         async function analyzeReview(event) {
#             event.preventDefault();
#             const review = document.getElementById('reviewInput').value.trim();
#             const resultDiv = document.getElementById('result');
#             const processedDiv = document.getElementById('processed');

#             if (!review) {
#                 resultDiv.innerHTML = '<p class="error">Please enter a review.</p>';
#                 processedDiv.innerHTML = '';
#                 return;
#             }

#             resultDiv.innerHTML = '<p>Analyzing...</p>';
#             processedDiv.innerHTML = '';

#             try {
#                 const response = await fetch('/analyze', {
#                     method: 'POST',
#                     headers: {
#                         'Content-Type': 'application/json',
#                     },
#                     body: JSON.stringify({ review: review }),
#                 });

#                 const data = await response.json();

#                 if (response.ok) {
#                     resultDiv.innerHTML = `<p class="success">Sentiment: ${data.sentiment}</p>`;
#                     processedDiv.innerHTML = `<p class="processed-text"><strong>Processed Review:</strong> ${data.processed_text}</p>`;
#                 } else {
#                     resultDiv.innerHTML = `<p class="error">Error: ${data.error}</p>`;
#                     processedDiv.innerHTML = '';
#                 }
#             } catch (error) {
#                 resultDiv.innerHTML = '<p class="error">Error: Unable to connect to the server.</p>';
#                 processedDiv.innerHTML = '';
#             }
#         }
#     </script>
# </body>
# </html>
# """

# # CSS content
# css_content = """
# * {
#     margin: 0;
#     padding: 0;
#     box-sizing: border-box;
# }

# body {
#     background-color: #ffd9e8;
#     color: #333;
#     font-family: Arial, sans-serif;
#     font-size: 17px;
# }

# .header {
#     width: 100%;
#     height: 100px;
#     background-color: #F564A9;
#     padding-top: 25px;
# }

# .header h2 {
#     text-transform: capitalize;
#     font-size: 33px;
#     text-align: center;
#     color: white;
# }

# .parent {
#     margin-top: 50px;
#     display: flex;
#     align-items: center;
#     justify-content: center;
# }

# .collection {
#     background-color: #F564A9;
#     width: 90%;
#     max-width: 600px;
#     min-height: 400px;
#     display: flex;
#     flex-direction: column;
#     justify-content: center;
#     align-items: center;
#     border-radius: 20px;
#     padding: 20px;
#     box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
# }

# .collection p {
#     text-transform: capitalize;
#     font-size: 20px;
#     text-align: center;
#     color: white;
#     margin-bottom: 20px;
# }

# .collection textarea {
#     background-color: #ffd9e8;
#     margin-top: 10px;
#     width: 100%;
#     min-height: 150px;
#     resize: vertical;
#     border: none;
#     border-radius: 10px;
#     padding: 15px;
#     font-size: 17px;
#     color: #333;
#     box-sizing: border-box;
# }

# .collection textarea::placeholder {
#     color: #c90865;
#     text-align: left;
#     opacity: 1;
# }

# .collection button {
#     width: 150px;
#     height: 40px;
#     border: none;
#     border-radius: 10px;
#     font-size: 20px;
#     color: #c90865;
#     background-color: #ffd9e8;
#     cursor: pointer;
#     transition: background-color 0.3s;
#     margin-top: 20px;
# }

# .result {
#     margin-top: 20px;
#     text-align: center;
#     min-height: 30px;
# }

# .processed {
#     margin-top: 10px;
#     text-align: center;
#     min-height: 30px;
# }

# .processed-text {
#     color: #fff;
#     font-size: 16px;
#     background-color: #c90865;
#     padding: 10px;
#     border-radius: 5px;
#     word-wrap: break-word;
#     max-width: 100%;
# }

# .success {
#     color: #28a745;
#     font-weight: bold;
# }

# .error {
#     color: #dc3545;
#     font-weight: bold;
# }
# """

# # Save CSS file
# os.makedirs('static', exist_ok=True)
# with open('static/style.css', 'w') as f:
#     f.write(css_content)

# # Initialize Flask app
# app = Flask(__name__)

# # Load model, vectorizer, and selector
# model = joblib.load('linear_svc_model.pkl')
# vectorizer = joblib.load('tfidf_vectorizer.pkl')
# selector = joblib.load('chi2_selector.pkl')

# # Flask routes
# @app.route('/', methods=['GET'])
# def index():
#     return render_template_string(html_template)

# @app.route('/analyze', methods=['POST'])
# def analyze():
#     data = request.get_json()
#     review = data.get('review', '').strip()
#     logging.info(f"Received review: {review}")
#     if not review:
#         return jsonify({'error': 'Please enter a review.'}), 400
#     processed_text = preprocess_text(review)
#     logging.info(f"Processed text: {processed_text}")
#     text_tfidf = vectorizer.transform([processed_text])
#     text_tfidf_selected = selector.transform(text_tfidf)
#     prediction = model.predict(text_tfidf_selected)[0]
#     logging.info(f"Prediction: {prediction}")
#     sentiment = "Positive 😄✔️" if prediction == "1" else "Negative 😔❌"
#     return jsonify({
#         'sentiment': sentiment,
#         'processed_text': processed_text
#     })

# # Function to kill process on port
# def kill_process_on_port(port):
#     for proc in psutil.process_iter(['pid', 'name']):
#         try:
#             for conn in proc.connections(kind='inet'):
#                 if conn.laddr.port == port:
#                     print(f"Killing process {proc.pid} on port {port}")
#                     proc.kill()
#                     return True
#         except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
#             continue
#     return False

# # Start ngrok and Flask server
# ngrok.set_auth_token('2x9G2X34klXZC2VDcx4AFxbvAPm_2DCdLrALRo27kMQxwq55u')
# PORT = 5001
# try:
#     ngrok.kill()
#     kill_process_on_port(PORT)
#     public_url = ngrok.connect(PORT).public_url
#     print(f"Public URL: {public_url}")
# except Exception as e:
#     print(f"Error starting ngrok: {e}")
#     print("Please ensure no other ngrok sessions are active and try again. Check https://dashboard.ngrok.com/agents")

# def run_app():
#     print(f"Starting Flask server on port {PORT}...")
#     app.run(host='0.0.0.0', port=PORT, use_reloader=False)
#     print(f"Flask server running on port {PORT}.")

# threading.Thread(target=run_app, daemon=True).start()

# # Check server status
# time.sleep(2)
# try:
#     response = requests.get(f"{public_url}")
#     print("Server is up and running!")
# except Exception as e:
#     print(f"Server check failed: {e}")
#     print("The Flask server may not have started correctly. Please restart the cell or runtime.")

# # Test specific input
# test_review = "movie was good"
# processed_test = preprocess_text(test_review)
# print(f"Processed test review: {processed_test}")
# test_tfidf = vectorizer.transform([processed_test])
# test_tfidf_selected = selector.transform(test_tfidf)
# test_prediction = model.predict(test_tfidf_selected)[0]
# print(f"Prediction for 'movie was good': {test_prediction}")
# print(f"Sentiment: {'Positive 😄✔️' if test_prediction == '1' else 'Negative 😔❌'}")

In [3]:
# import os
# import re
# import string
# import numpy as np
# import pandas as pd
# import nltk
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer, PorterStemmer
# import requests
# import tarfile
# from io import BytesIO
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_selection import SelectKBest, chi2
# from sklearn.svm import LinearSVC
# from sklearn.metrics import accuracy_score
# import joblib
# from flask import Flask, request, render_template_string, jsonify
# from pyngrok import ngrok
# import threading
# import time
# import psutil
# import shutil
# import logging

# # Set up logging
# logging.basicConfig(level=logging.INFO)

# # Download NLTK data
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('punkt_tab')

# # Download and extract dataset (same as notebook)
# url = "http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz"
# output_csv = "data_set.csv"
# print("Downloading dataset...")
# response = requests.get(url)
# if response.status_code != 200:
#     raise Exception("Failed to download the dataset")
# print("Extracting dataset...")
# tar = tarfile.open(fileobj=BytesIO(response.content), mode="r:gz")
# tar.extractall(path="movie_reviews")
# tar.close()

# # Load reviews and labels
# reviews = []
# labels = []
# base_dir = "movie_reviews/txt_sentoken"
# pos_dir = os.path.join(base_dir, "pos")
# for filename in os.listdir(pos_dir):
#     if filename.endswith(".txt"):
#         with open(os.path.join(pos_dir, filename), "r", encoding="utf-8") as file:
#             review = file.read().strip()
#             reviews.append(review)
#             labels.append(1)
# neg_dir = os.path.join(base_dir, "neg")
# for filename in os.listdir(neg_dir):
#     if filename.endswith(".txt"):
#         with open(os.path.join(neg_dir, filename), "r", encoding="utf-8") as file:
#             review = file.read().strip()
#             reviews.append(review)
#             labels.append(0)

# # Create CSV file
# print("Creating CSV file...")
# data = {"review": reviews, "sentiment": labels}
# df = pd.DataFrame(data)
# df.to_csv(output_csv, index=False, encoding="utf-8")
# print(f"Dataset saved to {output_csv}")
# shutil.rmtree("movie_reviews")
# print("Cleaned up temporary files")

# # Text preprocessing functions
# def remove_punctuation(text):
#     return text.translate(str.maketrans('', '', string.punctuation))

# def normalize_text(text):
#     normalization_map = {
#         'u': 'you',
#         'ur': 'your',
#         'r': 'are',
#         'b4': 'before',
#         'gr8': 'great',
#         'gud': 'good',
#         'pls': 'please',
#         'plz': 'please',
#         'thx': 'thanks',
#         'l8r': 'later',
#         'msg': 'message',
#         'btw': 'by the way',
#         'idk': 'i do not know',
#         'imo': 'in my opinion'
#     }
#     words = text.split()
#     return ' '.join([normalization_map.get(word, word) for word in words])

# def stem_text(text):
#     stemmer = PorterStemmer()
#     stop_words = set(stopwords.words('english')) - {'was'}
#     tokens = text.split()
#     return ' '.join([stemmer.stem(token) for token in tokens if token not in stop_words])

# def remove_word_duplicates(text):
#     words = word_tokenize(text)
#     seen = set()
#     new_words = []
#     for word in words:
#         if word.lower() not in seen:
#             seen.add(word.lower())
#             new_words.append(word)
#     return ' '.join(new_words)

# def preprocess_text(text):
#     text = text.lower()
#     text = remove_punctuation(text)
#     text = normalize_text(text)
#     text = ' '.join([word for word in text.split() if word not in (set(stopwords.words('english')) - {'was'})])
#     text = remove_word_duplicates(text)
#     lemmatizer = WordNetLemmatizer()
#     text = ' '.join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text)])
#     text = stem_text(text)
#     return text

# # Preprocess data
# df['review'] = df['review'].apply(preprocess_text)
# df['sentiment'] = df['sentiment'].astype(str)

# # Split data
# X = df['review'].astype(str).tolist()
# y = df['sentiment']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Vectorize and select features (aligned with traditional ML preprocessing from notebook)
# # Note: Using TF-IDF and chi2 feature selection as they are effective for LinearSVC and simpler than deep learning preprocessing (tokenization, padding, embeddings)
# vectorizer = TfidfVectorizer(max_features=10000)
# X_train_tfidf = vectorizer.fit_transform(X_train)
# X_test_tfidf = vectorizer.transform(X_test)

# selector = SelectKBest(chi2, k=5000)
# X_train_tfidf = selector.fit_transform(X_train_tfidf, y_train)
# X_test_tfidf = selector.transform(X_test_tfidf)

# # Train model (LinearSVC chosen based on notebook's conclusion that traditional models outperform deep learning models)
# # Notebook insight: Deep learning models (CNN, GRU) overfit, while LinearSVC generalizes better with simpler preprocessing
# model = LinearSVC(C=1, random_state=42, class_weight='balanced', max_iter=10000)
# model.fit(X_train_tfidf, y_train)

# # Evaluate model
# y_pred = model.predict(X_test_tfidf)
# print(f"Test Accuracy: {accuracy_score(y_test, y_pred)}")

# # Debug vocabulary and feature weights
# print("Vocabulary contains 'good':", "good" in vectorizer.vocabulary_)
# print("Vocabulary contains 'movi':", "movi" in vectorizer.vocabulary_)
# selected_features = vectorizer.get_feature_names_out()[selector.get_support()]
# print("Selected features contain 'good':", "good" in selected_features)
# print("Selected features contain 'movi':", "movi" in selected_features)
# feature_weights = model.coef_[0]
# feature_importance = dict(zip(selected_features, feature_weights))
# print("Weight for 'good':", feature_importance.get("good", "Not found"))
# print("Weight for 'movi':", feature_importance.get("movi", "Not found"))

# # Save model, vectorizer, and selector
# joblib.dump(model, 'linear_svc_model.pkl')
# joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
# joblib.dump(selector, 'chi2_selector.pkl')
# print("Model, vectorizer, and selector saved.")

# # HTML template (unchanged)
# html_template = """
# <!DOCTYPE html>
# <html lang="en">
# <head>
#     <meta charset="UTF-8">
#     <meta name="viewport" content="width=device-width, initial-scale=1.0">
#     <title>Sentiment Analysis</title>
#     <link rel="stylesheet" href="/static/style.css">
# </head>
# <body>
#     <div class="header">
#         <h2>Sentiment Analysis of Movie Reviews</h2>
#     </div>
#     <div class="parent">
#         <div class="collection">
#             <p>Please enter your text in English for analysis</p>
#             <form onsubmit="analyzeReview(event)">
#                 <textarea id="reviewInput" placeholder="Type here..." cols="70" rows="10"></textarea>
#                 <button type="submit">Analyze</button>
#             </form>
#             <div id="result" class="result"></div>
#             <div id="processed" class="processed"></div>
#         </div>
#     </div>

#     <script>
#         // Auto-resize textarea
#         const textarea = document.getElementById('reviewInput');
#         textarea.addEventListener('input', function () {
#             this.style.height = 'auto';
#             this.style.height = `${this.scrollHeight}px`;
#         });

#         // Handle form submission and send request to backend
#         async function analyzeReview(event) {
#             event.preventDefault();
#             const review = document.getElementById('reviewInput').value.trim();
#             const resultDiv = document.getElementById('result');
#             const processedDiv = document.getElementById('processed');

#             if (!review) {
#                 resultDiv.innerHTML = '<p class="error">Please enter a review.</p>';
#                 processedDiv.innerHTML = '';
#                 return;
#             }

#             resultDiv.innerHTML = '<p>Analyzing...</p>';
#             processedDiv.innerHTML = '';

#             try {
#                 const response = await fetch('/analyze', {
#                     method: 'POST',
#                     headers: {
#                         'Content-Type': 'application/json',
#                     },
#                     body: JSON.stringify({ review: review }),
#                 });

#                 const data = await response.json();

#                 if (response.ok) {
#                     resultDiv.innerHTML = `<p class="success">Sentiment: ${data.sentiment}</p>`;
#                     processedDiv.innerHTML = `<p class="processed-text"><strong>Processed Review:</strong> ${data.processed_text}</p>`;
#                 } else {
#                     resultDiv.innerHTML = `<p class="error">Error: ${data.error}</p>`;
#                     processedDiv.innerHTML = '';
#                 }
#             } catch (error) {
#                 resultDiv.innerHTML = '<p class="error">Error: Unable to connect to the server.</p>';
#                 processedDiv.innerHTML = '';
#             }
#         }
#     </script>
# </body>
# </html>
# """

# # CSS content (unchanged)
# css_content = """
# * {
#     margin: 0;
#     padding: 0;
#     box-sizing: border-box;
# }

# body {
#     background-color: #ffd9e8;
#     color: #333;
#     font-family: Arial, sans-serif;
#     font-size: 17px;
# }

# .header {
#     width: 100%;
#     height: 100px;
#     background-color: #F564A9;
#     padding-top: 25px;
# }

# .header h2 {
#     text-transform: capitalize;
#     font-size: 33px;
#     text-align: center;
#     color: white;
# }

# .parent {
#     margin-top: 50px;
#     display: flex;
#     align-items: center;
#     justify-content: center;
# }

# .collection {
#     background-color: #F564A9;
#     width: 90%;
#     max-width: 600px;
#     min-height: 400px;
#     display: flex;
#     flex-direction: column;
#     justify-content: center;
#     align-items: center;
#     border-radius: 20px;
#     padding: 20px;
#     box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
# }

# .collection p {
#     text-transform: capitalize;
#     font-size: 20px;
#     text-align: center;
#     color: white;
#     margin-bottom: 20px;
# }

# .collection textarea {
#     background-color: #ffd9e8;
#     margin-top: 10px;
#     width: 100%;
#     min-height: 150px;
#     resize: vertical;
#     border: none;
#     border-radius: 10px;
#     padding: 15px;
#     font-size: 17px;
#     color: #333;
#     box-sizing: border-box;
# }

# .collection textarea::placeholder {
#     color: #c90865;
#     text-align: left;
#     opacity: 1;
# }

# .collection button {
#     width: 150px;
#     height: 40px;
#     border: none;
#     border-radius: 10px;
#     font-size: 20px;
#     color: #c90865;
#     background-color: #ffd9e8;
#     cursor: pointer;
#     transition: background-color 0.3s;
#     margin-top: 20px;
# }

# .result {
#     margin-top: 20px;
#     text-align: center;
#     min-height: 30px;
# }

# .processed {
#     margin-top: 10px;
#     text-align: center;
#     min-height: 30px;
# }

# .processed-text {
#     color: #fff;
#     font-size: 16px;
#     background-color: #c90865;
#     padding: 10px;
#     border-radius: 5px;
#     word-wrap: break-word;
#     max-width: 100%;
# }

# .success {
#     color: #28a745;
#     font-weight: bold;
# }

# .error {
#     color: #dc3545;
#     font-weight: bold;
# }
# """

# # Save CSS file
# os.makedirs('static', exist_ok=True)
# with open('static/style.css', 'w') as f:
#     f.write(css_content)

# # Initialize Flask app
# app = Flask(__name__)

# # Load model, vectorizer, and selector
# model = joblib.load('linear_svc_model.pkl')
# vectorizer = joblib.load('tfidf_vectorizer.pkl')
# selector = joblib.load('chi2_selector.pkl')

# # Flask routes
# @app.route('/', methods=['GET'])
# def index():
#     return render_template_string(html_template)

# @app.route('/analyze', methods=['POST'])
# def analyze():
#     data = request.get_json()
#     review = data.get('review', '').strip()
#     logging.info(f"Received review: {review}")
#     if not review:
#         return jsonify({'error': 'Please enter a review.'}), 400
#     processed_text = preprocess_text(review)
#     logging.info(f"Processed text: {processed_text}")
#     text_tfidf = vectorizer.transform([processed_text])
#     text_tfidf_selected = selector.transform(text_tfidf)
#     prediction = model.predict(text_tfidf_selected)[0]
#     logging.info(f"Prediction: {prediction}")
#     sentiment = "Positive 😄✔️" if prediction == "1" else "Negative 😔❌"
#     return jsonify({
#         'sentiment': sentiment,
#         'processed_text': processed_text
#     })

# # Function to kill process on port
# def kill_process_on_port(port):
#     for proc in psutil.process_iter(['pid', 'name']):
#         try:
#             for conn in proc.connections(kind='inet'):
#                 if conn.laddr.port == port:
#                     print(f"Killing process {proc.pid} on port {port}")
#                     proc.kill()
#                     return True
#         except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
#             continue
#     return False

# # Start ngrok and Flask server
# ngrok.set_auth_token('2x9G2X34klXZC2VDcx4AFxbvAPm_2DCdLrALRo27kMQxwq55u')
# PORT = 5001
# try:
#     ngrok.kill()
#     kill_process_on_port(PORT)
#     public_url = ngrok.connect(PORT).public_url
#     print(f"Public URL: {public_url}")
# except Exception as e:
#     print(f"Error starting ngrok: {e}")
#     print("Please ensure no other ngrok sessions are active and try again. Check https://dashboard.ngrok.com/agents")

# def run_app():
#     print(f"Starting Flask server on port {PORT}...")
#     app.run(host='0.0.0.0', port=PORT, use_reloader=False)
#     print(f"Flask server running on port {PORT}.")

# threading.Thread(target=run_app, daemon=True).start()

# # Check server status
# time.sleep(2)
# try:
#     response = requests.get(f"{public_url}")
#     print("Server is up and running!")
# except Exception as e:
#     print(f"Server check failed: {e}")
#     print("The Flask server may not have started correctly. Please restart the cell or runtime.")

# # Test specific input
# test_review = "movie was good"
# processed_test = preprocess_text(test_review)
# print(f"Processed test review: {processed_test}")
# test_tfidf = vectorizer.transform([processed_test])
# test_tfidf_selected = selector.transform(test_tfidf)
# test_prediction = model.predict(test_tfidf_selected)[0]
# print(f"Prediction for 'movie was good': {test_prediction}")
# print(f"Sentiment: {'Positive 😄✔️' if test_prediction == '1' else 'Negative 😔❌'}")

In [4]:
import os
import re
import string
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import requests
import tarfile
from io import BytesIO
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
import joblib
from flask import Flask, request, render_template_string, jsonify
from pyngrok import ngrok
import threading
import psutil
import shutil
import logging

# Set up logging for Flask/Werkzeug to suppress INFO logs
logging.getLogger('werkzeug').setLevel(logging.WARNING)  # Changed from INFO to WARNING
logging.basicConfig(level=logging.INFO)

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

# Download and extract dataset
url = "http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz"
output_csv = "data_set.csv"
print("Downloading dataset...")
response = requests.get(url)
if response.status_code != 200:
    raise Exception("Failed to download the dataset")
print("Extracting dataset...")
tar = tarfile.open(fileobj=BytesIO(response.content), mode="r:gz")
tar.extractall(path="movie_reviews")
tar.close()

# Load reviews and labels
reviews = []
labels = []
base_dir = "movie_reviews/txt_sentoken"
pos_dir = os.path.join(base_dir, "pos")
for filename in os.listdir(pos_dir):
    if filename.endswith(".txt"):
        with open(os.path.join(pos_dir, filename), "r", encoding="utf-8") as file:
            review = file.read().strip()
            reviews.append(review)
            labels.append(1)
neg_dir = os.path.join(base_dir, "neg")
for filename in os.listdir(neg_dir):
    if filename.endswith(".txt"):
        with open(os.path.join(neg_dir, filename), "r", encoding="utf-8") as file:
            review = file.read().strip()
            reviews.append(review)
            labels.append(0)

# Create CSV file
print("Creating CSV file...")
data = {"review": reviews, "sentiment": labels}
df = pd.DataFrame(data)
df.to_csv(output_csv, index=False, encoding="utf-8")
print(f"Dataset saved to {output_csv}")
shutil.rmtree("movie_reviews")
print("Cleaned up temporary files")

# Text preprocessing functions
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def normalize_text(text):
    normalization_map = {
        'u': 'you',
        'ur': 'your',
        'r': 'are',
        'b4': 'before',
        'gr8': 'great',
        'gud': 'good',
        'pls': 'please',
        'plz': 'please',
        'thx': 'thanks',
        'l8r': 'later',
        'msg': 'message',
        'btw': 'by the way',
        'idk': 'i do not know',
        'imo': 'in my opinion'
    }
    words = text.split()
    return ' '.join([normalization_map.get(word, word) for word in words])

def stem_text(text):
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english')) - {'was'}
    tokens = text.split()
    return ' '.join([stemmer.stem(token) for token in tokens if token not in stop_words])

def remove_word_duplicates(text):
    words = word_tokenize(text)
    seen = set()
    new_words = []
    for word in words:
        if word.lower() not in seen:
            seen.add(word.lower())
            new_words.append(word)
    return ' '.join(new_words)

def preprocess_text(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = normalize_text(text)
    text = ' '.join([word for word in text.split() if word not in (set(stopwords.words('english')) - {'was'})])
    text = remove_word_duplicates(text)
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text)])
    text = stem_text(text)
    return text

# Preprocess data
df['review'] = df['review'].apply(preprocess_text)
df['sentiment'] = df['sentiment'].astype(str)

# Split data
X = df['review'].astype(str).tolist()
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize and select features
vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

selector = SelectKBest(chi2, k=5000)
X_train_tfidf = selector.fit_transform(X_train_tfidf, y_train)
X_test_tfidf = selector.transform(X_test_tfidf)

# Train model
model = LinearSVC(C=1, random_state=42, class_weight='balanced', max_iter=10000)
model.fit(X_train_tfidf, y_train)

# Evaluate model
y_pred = model.predict(X_test_tfidf)
print(f"Test Accuracy: {accuracy_score(y_test, y_pred)}")

# Debug vocabulary and feature weights
print("Vocabulary contains 'good':", "good" in vectorizer.vocabulary_)
print("Vocabulary contains 'movi':", "movi" in vectorizer.vocabulary_)
selected_features = vectorizer.get_feature_names_out()[selector.get_support()]
print("Selected features contain 'good':", "good" in selected_features)
print("Selected features contain 'movi':", "movi" in selected_features)
feature_weights = model.coef_[0]
feature_importance = dict(zip(selected_features, feature_weights))
print("Weight for 'good':", feature_importance.get("good", "Not found"))
print("Weight for 'movi':", feature_importance.get("movi", "Not found"))

# Save model, vectorizer, and selector
joblib.dump(model, 'linear_svc_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
joblib.dump(selector, 'chi2_selector.pkl')
print("Model, vectorizer, and selector saved.")

# HTML template
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Sentiment Analysis</title>
    <link rel="stylesheet" href="/static/style.css">
</head>
<body>
    <div class="header">
        <h2>Sentiment Analysis of Movie Reviews</h2>
    </div>
    <div class="parent">
        <div class="collection">
            <p>Please enter your text in English for analysis</p>
            <form onsubmit="analyzeReview(event)">
                <textarea id="reviewInput" placeholder="Type here..." cols="70" rows="10"></textarea>
                <button type="submit">Analyze</button>
            </form>
            <div id="result" class="result"></div>
            <div id="processed" class="processed"></div>
        </div>
    </div>

    <script>
        // Auto-resize textarea
        const textarea = document.getElementById('reviewInput');
        textarea.addEventListener('input', function () {
            this.style.height = 'auto';
            this.style.height = `${this.scrollHeight}px`;
        });

        // Handle form submission and send request to backend
        async function analyzeReview(event) {
            event.preventDefault();
            const review = document.getElementById('reviewInput').value.trim();
            const resultDiv = document.getElementById('result');
            const processedDiv = document.getElementById('processed');

            if (!review) {
                resultDiv.innerHTML = '<p class="error">Please enter a review.</p>';
                processedDiv.innerHTML = '';
                return;
            }

            resultDiv.innerHTML = '<p>Analyzing...</p>';
            processedDiv.innerHTML = '';

            try {
                const response = await fetch('/analyze', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify({ review: review }),
                });

                const data = await response.json();

                if (response.ok) {
                    resultDiv.innerHTML = `<p class="success">Sentiment: ${data.sentiment}</p>`;
                    processedDiv.innerHTML = `<p class="processed-text"><strong>Processed Review:</strong> ${data.processed_text}</p>`;
                } else {
                    resultDiv.innerHTML = `<p class="error">Error: ${data.error}</p>`;
                    processedDiv.innerHTML = '';
                }
            } catch (error) {
                resultDiv.innerHTML = `<p class="error">Error: Unable to connect to the server.</p>`;
                processedDiv.innerHTML = '';
            }
        }
    </script>
</body>
</html>
"""

# CSS content
css_content = """
* {
    margin: 0;
    padding: 0;
    box-sizing: border-box;
}

body {
    background-color: #ffd9e8;
    color: #333;
    font-family: Arial, sans-serif;
    font-size: 17px;
}

.header {
    width: 100%;
    height: 100px;
    background-color: #F564A9;
    padding-top: 25px;
}

.header h2 {
    text-transform: capitalize;
    font-size: 33px;
    text-align: center;
    color: white;
}

.parent {
    margin-top: 50px;
    display: flex;
    align-items: center;
    justify-content: center;
}

.collection {
    background-color: #F564A9;
    width: 90%;
    max-width: 600px;
    min-height: 400px;
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    border-radius: 20px;
    padding: 20px;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
}

.collection p {
    text-transform: capitalize;
    font-size: 20px;
    text-align: center;
    color: white;
    margin-bottom: 20px;
}

.collection textarea {
    background-color: #ffd9e8;
    margin-top: 10px;
    width: 100%;
    min-height: 150px;
    resize: vertical;
    border: none;
    border-radius: 10px;
    padding: 15px;
    font-size: 17px;
    color: #333;
    box-sizing: border-box;
}

.collection textarea::placeholder {
    color: #c90865;
    text-align: left;
    opacity: 1;
}

.collection button {
    width: 150px;
    height: 40px;
    border: none;
    border-radius: 10px;
    font-size: 20px;
    color: #c90865;
    background-color: #ffd9e8;
    cursor: pointer;
    transition: background-color 0.3s;
    margin-top: 20px;
}

.result {
    margin-top: 20px;
    text-align: center;
    min-height: 30px;
}

.processed {
    margin-top: 10px;
    text-align: center;
    min-height: 30px;
}

.processed-text {
    color: #fff;
    font-size: 16px;
    background-color: #c90865;
    padding: 10px;
    border-radius: 5px;
    word-wrap: break-word;
    max-width: 100%;
}

.success {
    color: #28a745;
    font-weight: bold;
}

.error {
    color: #dc3545;
    font-weight: bold;
}
"""

# Save CSS file
os.makedirs('static', exist_ok=True)
with open('static/style.css', 'w') as f:
    f.write(css_content)

# Initialize Flask app
app = Flask(__name__)

# Load model, vectorizer, and selector
model = joblib.load('linear_svc_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')
selector = joblib.load('chi2_selector.pkl')

# Flask routes
@app.route('/', methods=['GET'])
def index():
    return render_template_string(html_template)

@app.route('/analyze', methods=['POST'])
def analyze():
    data = request.get_json()
    review = data.get('review', '').strip()
    logging.info(f"Received review: {review}")
    if not review:
        return jsonify({'error': 'Please enter a review.'}), 400
    processed_text = preprocess_text(review)
    logging.info(f"Processed text: {processed_text}")
    text_tfidf = vectorizer.transform([processed_text])
    text_tfidf_selected = selector.transform(text_tfidf)
    prediction = model.predict(text_tfidf_selected)[0]
    logging.info(f"Prediction: {prediction}")
    sentiment = "Positive 😄✔️" if prediction == "1" else "Negative 😔❌"
    return jsonify({
        'sentiment': sentiment,
        'processed_text': processed_text
    })

# Function to kill process on port
def kill_process_on_port(port):
    for proc in psutil.process_iter(['pid', 'name']):
        try:
            for conn in proc.connections(kind='inet'):
                if conn.laddr.port == port:
                    print(f"Killing process {proc.pid} on port {port}")
                    proc.kill()
                    return True
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            continue
    return False

# Start ngrok and Flask server
ngrok.set_auth_token('2x9G2X34klXZC2VDcx4AFxbvAPm_2DCdLrALRo27kMQxwq55u')
PORT = 5001
try:
    ngrok.kill()
    kill_process_on_port(PORT)
    public_url = ngrok.connect(PORT).public_url
    print(f"Public URL: {public_url}")
except Exception as e:
    print(f"Error starting ngrok: {e}")
    print("Please ensure no other ngrok sessions are active and try again. Check https://dashboard.ngrok.com/agents")

def run_app():
    print(f"Starting Flask server on port {PORT}...")
    app.run(host='0.0.0.0', port=PORT, use_reloader=False)

threading.Thread(target=run_app, daemon=True).start()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Extracting dataset...
Creating CSV file...
Dataset saved to data_set.csv
Cleaned up temporary files
Test Accuracy: 0.85
Vocabulary contains 'good': True
Vocabulary contains 'movi': True
Selected features contain 'good': False
Selected features contain 'movi': False
Weight for 'good': Not found
Weight for 'movi': Not found
Model, vectorizer, and selector saved.
Public URL: https://078e-34-82-214-188.ngrok-free.app
Starting Flask server on port 5001...
 * Serving Flask app '__main__'
 * Debug mode: off
